<a href="https://colab.research.google.com/github/tig1795/word_embeddings_Lexika/blob/main/Bert_bert_base_german_cased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 44.6 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 895 kB 45.3 MB/s 
     |████████████████████████████████| 6.5 MB 41.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-german-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/249k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/474k [00:00<?, ?B/s]

In [ ]:
text = "Bank"
marked_text = "[CLS] " + text + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)

# Print out the tokens.
print (tokenized_text)

['[CLS]', 'Bank', '[SEP]']


In [ ]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

In [ ]:
indexed_tokens

[3, 2565, 4]

In [ ]:
segments_ids = [1] * len(tokenized_text)

In [ ]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

# Load pre-trained model (weights)
model = AutoModel.from_pretrained('bert-base-german-cased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()


Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
tokens_tensor

tensor([[   3, 2565,    4]])

In [ ]:
# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)

In [ ]:
encoded_layers

'last_hidden_state'

In [ ]:
from transformers import BertTokenizer, BertModel

import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-german-cased')

model = BertModel.from_pretrained('bert-base-german-cased')

word = "Bank"

inputs = tokenizer(word, return_tensors="pt")

outputs = model(**inputs)

word_vect = outputs.pooler_output.detach().numpy()

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
w1 = model(**tokenizer("bank", return_tensors="pt")).pooler_output.detach().numpy()
w2 = model(**tokenizer("bank", return_tensors="pt")).pooler_output.detach().numpy()

In [ ]:
w2.shape

(1, 768)

In [ ]:
w1.shape

(1, 768)

In [ ]:
from numpy import dot
from numpy.linalg import norm

a = w1
b = w2.T

cos_sim = dot(a, b)/(norm(a)*norm(b))


In [ ]:
cos_sim[0][0]

1.0

In [ ]:
import operator
def foo(w, l):
  w1 = model(**tokenizer(w, return_tensors="pt")).pooler_output.detach().numpy()

  output = {}

  for tw in l:
    w2 = model(**tokenizer(tw, return_tensors="pt")).pooler_output.detach().numpy()
    w2 = w2.T
    cos_sim = dot(w1, w2)/(norm(w1)*norm(w2))
    output[tw] = cos_sim[0][0] # schlauer mit np.squeeze

  return sorted(output.items(), key=lambda kv: kv[1], reverse=True)

## Versuch mit ausgepowert

In [ ]:
w = "ausgepowert"

power = foo(w, tokenizer.vocab)

In [ ]:
power

[('##Preis', 0.9542382),
 ('gekauft', 0.9447617),
 ('##preisträger', 0.9435616),
 ('versprochen', 0.94270235),
 ('##verbrauch', 0.9424161),
 ('##Aktien', 0.93914044),
 ('##verlust', 0.9390882),
 ('##gewinn', 0.9389764),
 ('geeinigt', 0.93868226),
 ('##förder', 0.9379542),
 ('##Markt', 0.9368774),
 ('##gewachsen', 0.9368571),
 ('##geschlagen', 0.93643177),
 ('gespalten', 0.93629897),
 ('##gesehen', 0.9362758),
 ('Kurs', 0.93605655),
 ('##gelöst', 0.93597144),
 ('##Mittel', 0.9357279),
 ('##Unternehmen', 0.9355875),
 ('##angelegenheiten', 0.93545973),
 ('##wertung', 0.9354251),
 ('##gebracht', 0.9353236),
 ('##umsatz', 0.9352334),
 ('##gelegt', 0.9351039),
 ('##wertig', 0.9345459),
 ('gerechnet', 0.9342104),
 ('##gesetzt', 0.9340554),
 ('##hoch', 0.9336597),
 ('gesunken', 0.93358314),
 ('##gerechnet', 0.93344134),
 ('##anal', 0.9333),
 ('##Geschäft', 0.9332565),
 ('##wert', 0.93319845),
 ('##erhalt', 0.93301594),
 ('kräftig', 0.93301445),
 ('attraktiv', 0.9328089),
 ('##kalk', 0.93277985

# Versuch mit Feminismus

In [ ]:
w = "Feminismus"

Hi = foo(w, tokenizer.vocab)

In [ ]:
Hi

[('Journalismus', 0.9320039),
 ('Islam', 0.9153972),
 ('Ethik', 0.9138482),
 ('Intellekt', 0.9127552),
 ('Kapitalismus', 0.9097322),
 ('Rassismus', 0.90690005),
 ('Identität', 0.9068035),
 ('Tierschutz', 0.9052345),
 ('Lehre', 0.90509796),
 ('Philosophie', 0.9040371),
 ('Christentum', 0.90383524),
 ('Toleranz', 0.9032742),
 ('Reflex', 0.9026124),
 ('Verhalten', 0.9022104),
 ('##ifizierung', 0.90194094),
 ('Christen', 0.9017803),
 ('Ökonomie', 0.9015981),
 ('Lobby', 0.90008956),
 ('Religion', 0.8995289),
 ('Kontext', 0.8992756),
 ('Meinungs', 0.8991688),
 ('Intervention', 0.89913595),
 ('Standpunkt', 0.8990546),
 ('Haltung', 0.8989758),
 ('Konsens', 0.8988254),
 ('Evolution', 0.8984677),
 ('Glauben', 0.8980897),
 ('Politische', 0.89781255),
 ('sexuelle', 0.89778465),
 ('##izismus', 0.89633685),
 ('Report', 0.89611125),
 ('Analyse', 0.8955748),
 ('Kulturen', 0.8952659),
 ('Werbung', 0.89498216),
 ('Opposition', 0.89415294),
 ('Aspekt', 0.89411),
 ('Meinung', 0.8938122),
 ('Sexual', 0.893

Ergebnis ist nicht gut, die angegebenen ähnlichsten Wörter haben meistens nichts mit Feminismus zu tun.

# Versuch mit blöde

In [ ]:
r = "blöde"

blo = foo(r, tokenizer.vocab)

In [ ]:
blo

[('##klusive', 0.9506972),
 ('##ätzliche', 0.94941103),
 ('##samkeit', 0.94874734),
 ('##alerweise', 0.9478096),
 ('##unschwei', 0.94720453),
 ('##icherheit', 0.9463063),
 ('##fassungs', 0.9456839),
 ('##ändige', 0.9450107),
 ('##fassungsgericht', 0.9449849),
 ('##eigner', 0.94420195),
 ('##losigkeit', 0.94377553),
 ('##ändiger', 0.9433945),
 ('##lägerin', 0.9432407),
 ('##worter', 0.94305205),
 ('##ändig', 0.9427977),
 ('##lässigkeit', 0.9418709),
 ('##öge', 0.9417256),
 ('##hörige', 0.94155556),
 ('##parade', 0.94128835),
 ('##anspruch', 0.94105667),
 ('##atische', 0.941043),
 ('##hörigkeit', 0.94101053),
 ('##lehnung', 0.94008225),
 ('##ziffer', 0.9392511),
 ('##kapelle', 0.9391625),
 ('##mantel', 0.9388861),
 ('##ängliche', 0.9387095),
 ('##ners', 0.93845063),
 ('##ahnung', 0.93843347),
 ('##bard', 0.93819624),
 ('##laude', 0.9379142),
 ('##ertal', 0.9379098),
 ('##rüder', 0.9378103),
 ('##stunde', 0.9378078),
 ('##mädchen', 0.9373639),
 ('##platte', 0.93734574),
 ('##zige', 0.9370

# Versuch mit Eselsbrücke

In [ ]:
r = "Eselsbrücke"

esel = foo(r, tokenizer.vocab)

In [ ]:
esel

[('Brücke', 0.9512461),
 ('Trasse', 0.922187),
 ('Elbe', 0.91291356),
 ('Autobahn', 0.91191447),
 ('Bridge', 0.9107263),
 ('Kapelle', 0.91010106),
 ('Hügel', 0.90850586),
 ('Stadtmauer', 0.90668833),
 ('Brunnen', 0.90568054),
 ('Turm', 0.9055144),
 ('Ruine', 0.9033671),
 ('Treppe', 0.90207046),
 ('Spree', 0.9017582),
 ('Kanzel', 0.90049875),
 ('Ostsee', 0.8990718),
 ('##Kanal', 0.8984447),
 ('Kanal', 0.8978636),
 ('Höhle', 0.89723057),
 ('Havel', 0.89632404),
 ('Burg', 0.8942327),
 ('Rhein', 0.8938047),
 ('Ufer', 0.8933599),
 ('Brücken', 0.8931806),
 ('Alm', 0.89316607),
 ('Bodensee', 0.8925798),
 ('Tunnel', 0.8922456),
 ('Moschee', 0.891724),
 ('See', 0.8907825),
 ('Fließ', 0.8885416),
 ('Küste', 0.8870771),
 ('Felsen', 0.88629526),
 ('Pier', 0.88615304),
 ('Landstraße', 0.88547045),
 ('Bundesstraße', 0.8847713),
 ('Kirche', 0.8847467),
 ('Kreuzung', 0.8847175),
 ('Höhlen', 0.8843631),
 ('Weser', 0.88429767),
 ('Bucht', 0.8835561),
 ('Route', 0.8833473),
 ('Sakristei', 0.88329834),
 (

# Versuch mit Toilette

In [ ]:
z = "Toilette"

toi = foo(z, tokenizer.vocab)

In [ ]:
toi

[('Küche', 0.9309593),
 ('Bett', 0.92625374),
 ('Tür', 0.9205587),
 ('Toiletten', 0.91998327),
 ('Toil', 0.9199125),
 ('Büro', 0.9143149),
 ('Supermarkt', 0.91060495),
 ('Apotheke', 0.9086891),
 ('Balkon', 0.90714335),
 ('Gerät', 0.9065315),
 ('Pflege', 0.90566224),
 ('Kleidung', 0.90528834),
 ('Teilzeit', 0.9040696),
 ('Tuch', 0.90392345),
 ('Wasch', 0.90371674),
 ('Fahrt', 0.90316725),
 ('Sau', 0.9006248),
 ('Fahrrad', 0.89939433),
 ('Kredit', 0.8992041),
 ('Boden', 0.89899874),
 ('Schwangerschaft', 0.8983924),
 ('##spielraum', 0.8982068),
 ('Mikro', 0.8981876),
 ('Wände', 0.8969523),
 ('Futter', 0.8963002),
 ('Speise', 0.89618295),
 ('##einrichtung', 0.8950452),
 ('Wohnzimmer', 0.8947485),
 ('Angestellte', 0.89471877),
 ('Wohnungs', 0.8942783),
 ('Fenster', 0.89393413),
 ('Nase', 0.8939153),
 ('Küchen', 0.8935979),
 ('##ausstrahlung', 0.8935291),
 ('##überwachung', 0.8928092),
 ('Kab', 0.8926228),
 ('Filter', 0.8923397),
 ('Taxi', 0.8919525),
 ('Wohnung', 0.89191085),
 ('##ifizierun

# Versuch mit Batterie

In [ ]:
h = "Batterie"

batt = foo(h, tokenizer.vocab)

In [ ]:
batt

[('Batterie', 1.0),
 ('diesel', 0.9437215),
 ('Tank', 0.94215125),
 ('Speicher', 0.9375412),
 ('Waffe', 0.9324351),
 ('Elektronik', 0.92920995),
 ('Prototyp', 0.92912495),
 ('Baureihe', 0.9285437),
 ('Erdgas', 0.92427754),
 ('Signal', 0.92409486),
 ('Autob', 0.9212961),
 ('Lade', 0.9212149),
 ('Elektro', 0.92058736),
 ('Marine', 0.92030805),
 ('elektrische', 0.9202407),
 ('Quad', 0.9185043),
 ('Radar', 0.91847897),
 ('Micro', 0.9178629),
 ('Laser', 0.9177788),
 ('Brigade', 0.91726667),
 ('Plattform', 0.9171696),
 ('Stadtbahn', 0.91712314),
 ('Soft', 0.9170824),
 ('Draht', 0.91678494),
 ('basiert', 0.916566),
 ('standard', 0.9164302),
 ('Systems', 0.9162079),
 ('Container', 0.91588473),
 ('Maschine', 0.9158379),
 ('Rail', 0.9157823),
 ('Smart', 0.9157728),
 ('Cab', 0.91567695),
 ('AT', 0.9155847),
 ('Artillerie', 0.9152518),
 ('Solar', 0.91518176),
 ('Flüssigkeit', 0.91430426),
 ('Elektrom', 0.91429126),
 ('Elektriz', 0.9141912),
 ('Turb', 0.9134569),
 ('Modell', 0.91341776),
 ('Dampf',

Interessant: Die Einträge ('Waffe', 0.9324351) und ('Marine', 0.92030805) deuten auf die ältere Bedeutung des Wortes Batterei hin, obwohl hier aktuelle Wikipediaartikel verwendet werden. 
Könnte guter Kandidat für Wortbedeutungswandel sein!

# Versuch mit Panzer

In [ ]:
r = "Panzer"

panz = foo(r, tokenizer.vocab)

In [ ]:
panz

[('Panzer', 0.99999994),
 ('Hubschrauber', 0.9237421),
 ('Zerstörer', 0.9223249),
 ('Kreuzer', 0.9182645),
 ('Baureihe', 0.9174612),
 ('Triebwagen', 0.9170865),
 ('Trieb', 0.914093),
 ('Boot', 0.91158307),
 ('Infanterie', 0.90896344),
 ('Bug', 0.90623635),
 ('Flugzeug', 0.9055342),
 ('Fiat', 0.90537983),
 ('Lokomotive', 0.90403765),
 ('Korps', 0.90301996),
 ('Boeing', 0.90190816),
 ('Maschinen', 0.90162426),
 ('RAF', 0.9008999),
 ('Reichsbahn', 0.9007994),
 ('Motorrad', 0.8979202),
 ('Lok', 0.8977568),
 ('ICE', 0.8970468),
 ('Wehrmacht', 0.8955071),
 ('Coup', 0.8954127),
 ('Stab', 0.895094),
 ('Opel', 0.8944699),
 ('diesel', 0.89359176),
 ('Wagen', 0.893221),
 ('Maz', 0.8928097),
 ('Bahn', 0.8922379),
 ('Schiff', 0.89218247),
 ('Mercedes', 0.891503),
 ('Heer', 0.8909705),
 ('Exemplar', 0.89030445),
 ('Kessel', 0.8898771),
 ('Gleis', 0.8896922),
 ('Fahrzeug', 0.8896859),
 ('Bauart', 0.8892969),
 ('Blech', 0.88899153),
 ('Abgas', 0.8889629),
 ('Kanonen', 0.88884026),
 ('Batterie', 0.8886

# Versuch mit wichsen

In [ ]:
k = "wichsen"

wich = foo(k, tokenizer.vocab)

In [ ]:
wich

[('##ieß', 0.96555096),
 ('##ze', 0.96351534),
 ('##ue', 0.9617319),
 ('##utze', 0.96000355),
 ('##unte', 0.9595246),
 ('##scha', 0.9569505),
 ('##ürmen', 0.95620644),
 ('##öhm', 0.9556325),
 ('##erei', 0.95492166),
 ('##örder', 0.95491415),
 ('##ische', 0.9544991),
 ('##ithm', 0.95431805),
 ('##ierenden', 0.95390993),
 ('##lichkeits', 0.95378315),
 ('##zes', 0.95371646),
 ('##oe', 0.9532393),
 ('##schungs', 0.95322704),
 ('##ilm', 0.95295924),
 ('##ogels', 0.95266443),
 ('##ilden', 0.9521939),
 ('##hus', 0.95184505),
 ('##iefe', 0.9516702),
 ('##waltungs', 0.95119786),
 ('##ilde', 0.95088685),
 ('##ommen', 0.9507088),
 ('##äuden', 0.95063853),
 ('##zen', 0.9505684),
 ('##ützten', 0.9503472),
 ('##ahls', 0.95034534),
 ('##ürme', 0.9502352),
 ('##ochte', 0.95018727),
 ('##uge', 0.94999397),
 ('##eise', 0.9499449),
 ('##ödien', 0.94964427),
 ('##otte', 0.9495171),
 ('##vergenz', 0.94930494),
 ('##äck', 0.94898486),
 ('##linde', 0.94872206),
 ('##issheit', 0.9486232),
 ('##öch', 0.9485251

# Versuch mit Widerstand

In [ ]:
w = "Widerstand"

wid = foo(w, tokenizer.vocab)

In [ ]:
wid

[('Widerstand', 1.0),
 ('Widerstands', 0.933416),
 ('Kampf', 0.9246181),
 ('Appell', 0.9219137),
 ('Bündnis', 0.91899675),
 ('terror', 0.9181946),
 ('Töt', 0.91792107),
 ('Frieden', 0.9174246),
 ('Aufstand', 0.915521),
 ('rechtsextrem', 0.9153838),
 ('Feind', 0.9147691),
 ('Gefangenen', 0.91467196),
 ('Vollzug', 0.9141719),
 ('Intellekt', 0.9141187),
 ('politisch', 0.9139066),
 ('bekämpfen', 0.91371024),
 ('Anliegen', 0.9136421),
 ('Bomben', 0.9113797),
 ('Bekenntnis', 0.9109018),
 ('Parteitag', 0.9107992),
 ('Toleranz', 0.90983284),
 ('äußert', 0.9097588),
 ('Freundschaft', 0.9096544),
 ('Antisemit', 0.9090821),
 ('Kämpfe', 0.909032),
 ('lehnt', 0.9089286),
 ('feind', 0.90891635),
 ('islam', 0.9086951),
 ('Kämpfen', 0.9086932),
 ('Front', 0.90816337),
 ('durchgesetzt', 0.90783817),
 ('Krieg', 0.9077769),
 ('Friedens', 0.90718085),
 ('Sozialdemokrat', 0.9069718),
 ('inhaftiert', 0.9067888),
 ('protest', 0.9065091),
 ('Konsens', 0.90647733),
 ('geäußert', 0.906135),
 ('Regime', 0.905899

# Versuch mit Rauchwaren

In [ ]:
y = "Rauchwaren"

rau = foo(y, tokenizer.vocab)

In [ ]:
rau

[('Zigaretten', 0.9513302),
 ('Ware', 0.93520397),
 ('Tabak', 0.93398064),
 ('Waren', 0.933475),
 ('Waffen', 0.9333919),
 ('Textil', 0.92795753),
 ('Abfall', 0.92665184),
 ('Arzneimittel', 0.9235721),
 ('Lebensmittel', 0.92349166),
 ('Gegenstände', 0.9232702),
 ('##kraftwerke', 0.9197851),
 ('Stoffe', 0.91912204),
 ('Kleidung', 0.91894704),
 ('Eigentum', 0.9153661),
 ('Schuhe', 0.9134745),
 ('Schienen', 0.9124542),
 ('Herstellung', 0.9117074),
 ('Rechte', 0.9112894),
 ('Ketten', 0.91085696),
 ('handwerk', 0.9100713),
 ('Geräte', 0.90931195),
 ('##bewegungen', 0.90863293),
 ('##enträger', 0.90835345),
 ('Kfz', 0.9079136),
 ('Möbel', 0.9075195),
 ('Hilfsmittel', 0.90740204),
 ('Industrie', 0.90707064),
 ('Bomben', 0.90703326),
 ('Werbung', 0.9067147),
 ('Formen', 0.90641075),
 ('Handwerker', 0.905947),
 ('Gehäuse', 0.9058982),
 ('Produkte', 0.90580577),
 ('Abgaben', 0.9057413),
 ('Dingen', 0.9057099),
 ('Fracht', 0.90567994),
 ('Geschäfte', 0.9054025),
 ('Papier', 0.90512353),
 ('Zeichen